In [1]:
import pandas as pd

# URL of the Wikipedia page
url = "https://en.wikipedia.org/wiki/List_of_United_States_FIPS_codes_by_county"

# Read the table from the URL
tables = pd.read_html(url)

# Select the desired table
df = tables[1]

# Extract the desired columns
selected_columns = [0, 1, 2]
df = df.iloc[:, selected_columns]

# Rename the columns
new_column_names = ["Combined", "County_name", "State"]
df = df.rename(columns=dict(zip(df.columns, new_column_names)))

# Print the resulting DataFrame
print(df)


      Combined        County_name    State
0         1001     Autauga County  Alabama
1         1003     Baldwin County  Alabama
2         1005     Barbour County  Alabama
3         1007        Bibb County  Alabama
4         1009      Blount County  Alabama
...        ...                ...      ...
3238     56037  Sweetwater County  Wyoming
3239     56039       Teton County  Wyoming
3240     56041       Uinta County  Wyoming
3241     56043    Washakie County  Wyoming
3242     56045      Weston County  Wyoming

[3243 rows x 3 columns]


In [2]:
# Convert "Combined" column to string type
df['Combined'] = df['Combined'].astype(str)

# Create new columns "county_code" and "state_code"
df['county_code'] = df['Combined'].str[-3:]
df['state_code'] = df['Combined'].str[:-3].apply(lambda x: x.zfill(2))

# Print the resulting DataFrame
print(df)

     Combined        County_name    State county_code state_code
0        1001     Autauga County  Alabama         001         01
1        1003     Baldwin County  Alabama         003         01
2        1005     Barbour County  Alabama         005         01
3        1007        Bibb County  Alabama         007         01
4        1009      Blount County  Alabama         009         01
...       ...                ...      ...         ...        ...
3238    56037  Sweetwater County  Wyoming         037         56
3239    56039       Teton County  Wyoming         039         56
3240    56041       Uinta County  Wyoming         041         56
3241    56043    Washakie County  Wyoming         043         56
3242    56045      Weston County  Wyoming         045         56

[3243 rows x 5 columns]


In [3]:
import pandas as pd

# Select desired columns
selected_columns = ['county_code', 'state_code', 'County_name']

# Create a new DataFrame with selected columns
new_df = df[selected_columns]

In [4]:
new_df

,county_code,state_code,County_name
0,001,01,Autauga County
1,003,01,Baldwin County
2,005,01,Barbour County
3,007,01,Bibb County
4,009,01,Blount County
...,...,...,...
3238,037,56,Sweetwater County
3239,039,56,Teton County
3240,041,56,Uinta County
3241,043,56,Washakie County


In [5]:
test_df = new_df.head(10)

In [6]:
test_df

,county_code,state_code,County_name
0,001,01,Autauga County
1,003,01,Baldwin County
2,005,01,Barbour County
3,007,01,Bibb County
4,009,01,Blount County
5,011,01,Bullock County
6,013,01,Butler County
7,015,01,Calhoun County
8,017,01,Chambers County
9,019,01,Cherokee County


In [7]:
import pandas as pd
import requests

# API URL
api_url = "https://api.census.gov/data/2011/acs/acs5/profile"

# Read the input file (assuming it's a .tsv file)
input_file = new_df
input_df = new_df

data_list = []
county_names = {}  # Dictionary to store county names

for i in range(len(input_df)):
    state_code = input_df.loc[i, "state_code"]
    county_code = input_df.loc[i, "county_code"]

    # Construct the 'in' parameter with multiple values
    in_param = f"state:{state_code} county:{county_code}"

    # Construct the complete API URL with all parameters
    url = f"{api_url}?get=NAME,DP05_0037PE,DP05_0038PE,DP05_0044PE,DP05_0071PE,DP04_0046PE,DP02_0092PE,DP02_0064PE,DP02_0065PE,DP02_0066PE,DP03_0028PE,DP03_0035PE,DP03_0027PE,DP03_0030PE,DP03_0031PE,DP03_0119PE&for=tract:*&in=state:{state_code}&in=county:{county_code}&key=2fe10d91b388952346f4b64aaf864928e049bc86"

    # Make the API call for each county
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        data_list.extend(data[1:])  # Append the data rows

        # Store county name in the dictionary
        county_names[county_code] = data[1][0]
    else:
        print(f"Failed to fetch data for {input_df.loc[i, 'County_name']}")

# Convert the data list to DataFrame
df = pd.DataFrame(data_list, columns=["NAME"] + data[0][1:])

# Add the 'county_name' column based on county_code
df["county_name"] = df["NAME"].apply(lambda x: county_names.get(x.split(",")[-1].strip(), None))

# Define a dictionary to map the old column names to the new column names
column_names = {
    "DP05_0037PE": "% White",
    "DP05_0038PE": "% Black",
    "DP05_0044PE": "% Asian",
    "DP05_0071PE": "% Hispanic",
    "DP04_0046PE": "% Owner occupied housing",
    "DP02_0092PE": "% Foreign born",
    "DP02_0064PE": "% Bachelor's degree",
    "DP02_0065PE": "% Graduate or professional degree",
    "DP02_0066PE": "% High school graduate or higher",
    "DP03_0028PE": "% Service occupations",
    "DP03_0035PE": "% Manufacturing",
    "DP03_0027PE": "% Management, business, science, and arts occupations",
    "DP03_0030PE": "% Natural resources, construction, and maintenance occupations",
    "DP03_0031PE": "% Production, transportation, and material moving occupations",
    "DP03_0119PE": "% Poverty"
}


# Rename the columns in the DataFrame
df = df.rename(columns=column_names)

df.to_csv('2011_final_all_counties.csv', index = False)


# Print the modified DataFrame
print(df)


Failed to fetch data for Chugach Census Area[b]
Failed to fetch data for Copper River Census Area[b]
Failed to fetch data for Kusilvak Census Area[b]
Failed to fetch data for Eastern District[2]
Failed to fetch data for Manu'a District[2]
Failed to fetch data for Rose Atoll (Rose Island)[2]
Failed to fetch data for Swain's Island[2]
Failed to fetch data for Western District[2]
Failed to fetch data for Guam[3]
Failed to fetch data for Northern Islands Municipality[3]
Failed to fetch data for Rota Municipality
Failed to fetch data for Saipan Municipality
Failed to fetch data for Tinian Municipality
Failed to fetch data for Oglala Lakota County
Failed to fetch data for Baker Island[4][v]
Failed to fetch data for Howland Island[v]
Failed to fetch data for Jarvis Island[v]
Failed to fetch data for Johnston Atoll[v]
Failed to fetch data for Kingman Reef[v]
Failed to fetch data for Midway Islands[v]
Failed to fetch data for Navassa Island[v]
Failed to fetch data for Palmyra Atoll[v]
Failed to

In [8]:
df['period'] = 2011

df.rename(columns={'tract': 'geoid'}, inplace=True)


df['county_name'] = df['NAME'].str.extract(r',\s*([^,]+),')


df = df[['geoid', 'period'] + [col for col in df.columns if col not in ['geoid', 'period']]]


df = df.drop('NAME', axis=1)


df.to_csv('2011_final_all_counties.csv', index = False)